In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model,preprocessing
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import time
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from ml_metrics import rmse,rmsle

In [2]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self,trainOrTestOrBoth): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
            
        meanOfDemanda = self.train["Demanda_uni_equil"].values.mean().astype("float32")
        
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.loc[:,"DemandaGeneralMean"] = meanOfDemanda
            self.test2.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        #(self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe.head(2))
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train.head(2))
            display(self.test1.head(2))
            display(self.test2.head(2))
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train.head(2))
        display(self.test1.head(2))
        display(self.test2.head(2))
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()

In [3]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [4]:
parameterDict =       {"ValidationStart":10, 
 "ValidationEnd":11,
   "maxLag":3,
    "trainHdfPath":'../../input/train_full.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1_full.h5",
    "testHdfPath2":"../../input/test2_full.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 3, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1_full.h5', 'ValidationEnd': 11, 'testHdfPath2': '../../input/test2_full.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 10, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_f

In [5]:
FE.ReadCsv('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 2.1 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
2,2,10,2045,1,2831,4549769,32940
7,7,10,1612,1,2837,4414012,35305


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 2 to 6999247
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 81.0 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 79.2 MB
None


## Merge town and products..

In [6]:
townstate = pd.read_csv("../../input/town_state.csv", encoding='utf-8')
townstate['Town_ID']=townstate['Town'].str[:4]
states = townstate['State']
le = preprocessing.LabelEncoder()
townstate['State_ID']=le.fit_transform(states)
townstate = townstate.drop(['Town', 'State'], axis=1)
townstate = townstate.astype('uint16')
townstate[['State_ID']] =townstate[['State_ID']] .astype('uint8')
FE.train = pd.merge(FE.train, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test1 = pd.merge(FE.test1, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test2 = pd.merge(FE.test2, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()

98

In [7]:
products = pd.read_csv("../../input/producto_tabla.csv")
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

le = preprocessing.LabelEncoder()

products['Prod_name_ID']=le.fit_transform(products['short_name_processed'])
products['Brand_ID']=le.fit_transform(products['brand'])

products = products.drop(['short_name', 'brand', 'short_name_processed', 'NombreProducto'], axis=1)
products.fillna(value=0, inplace=True)
products[['pieces','Brand_ID']] = products[['pieces','Brand_ID']].astype('uint8')
products[['Producto_ID','weight', 'Prod_name_ID']] = products[['Producto_ID','weight', 'Prod_name_ID']].astype('uint16')
FE.train = pd.merge(FE.train, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test1 = pd.merge(FE.test1, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test2 = pd.merge(FE.test2, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()

14

## Check NA values here.

In [8]:
FE.train[FE.train.isnull().any(axis=1)]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


In [9]:
FE.test1[FE.test1.isnull().any(axis=1)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


In [10]:
FE.test2[FE.test2.isnull().any(axis=1)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


## Last State of Train, test1 and test2..

In [11]:
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3,2008,16,120,2,709,4
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4,2008,16,135,2,712,4


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4


## Convert to Log..

In [12]:
FE.train.loc[:,"Demanda_uni_equil"] = np.log1p (FE.train["Demanda_uni_equil"].values)
gc.collect()

248

## Split Train to test1 test2

In [13]:
#FE.SplitTrainToTestUsingValidationStart()

## Add DemandaNotEqualTheDifferenceOfVentaUniAndDev to model products delivered later..

In [14]:
FE.train.loc[:,"DemandaNotEqualTheDifferenceOfVentaUniAndDev"] = FE.train.Demanda_uni_equil.values < (
    FE.train.Venta_uni_hoy.values - FE.train.Dev_uni_proxima.values)
gc.collect()

0

## Delete Demanda = 0  and Venta = 0

In [15]:
FE.train = FE.train[(FE.train.Demanda_uni_equil.values != 0) & (FE.train.Venta_uni_hoy.values != 0)]
gc.collect()

35

In [16]:
FE.SaveDataFrameToHdf('both')

In [17]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,True
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,True


<class 'pandas.core.frame.DataFrame'>
Int64Index: 72843643 entries, 0 to 74180463
Data columns (total 18 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 0 to 3538384
Data columns (total 13 columns):
id              uint32
Semana          uint8
Agencia_ID      uint16
Canal_ID        uint8
Ruta_SAK        uint16
Cliente_ID      uint32
Producto_ID     uint16
Town_ID         uint16
State_ID        uint8
weight          uint16
pieces          uint8
Prod_name_ID    uint16
Brand_ID        uint8
dtypes: uint16(6), uint32(2), uint8(5)
memory usage: 111.4 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 3460865
Data columns (total 13 columns):
id              uint32
Semana          uint8
Agencia_ID      uint16
Canal_ID        uint8
Ruta_SAK        uint16
Cliente_ID      uint32
Producto_ID     uint16
Town_ID         uint16
State_ID        uint8
weight          uint16
pieces          uint8
Prod_name_ID    uint16
Brand_ID        uint8
dtypes: uint16(6), uint32(2), uint8(5)
memory usage: 108.9 MB
None


## Grouping for Lag0

In [18]:
configLag0Target1DeleteColumnsFalse = ConfigElements(0,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"]),
                                                       ("SBClRACh0_mean",[ ##brand id START
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClRA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SBClRCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClACh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClR0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SBClA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBClCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBCl0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SBR0_mean",[
                                                              "Brand_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SBA0_mean",[
                                                              "Brand_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBCh0_mean",[
                                                              "Brand_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBT0_mean",[
                                                              "Brand_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SBSt0_mean",[
                                                              "Brand_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SB0_mean",[
                                                              "Brand_ID"],
                                                          ["mean"])
                                                       
                                                       
                                                       ], "Lag0", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [19]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.069195
            681747     3306     2281       7               7.317818

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,True,NaN,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,True,NaN,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SPClRACh0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,NaN,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SB0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.685899
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,1.595374


Count of missing numbers after SB0_mean in validation part 1 in column Lag0 is 2
Count of missing numbers after SB0_mean in validation part 2 in column Lag0 is 3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,True,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,True,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384


In [20]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [21]:
FE.SaveDataFrameToHdf('both')

In [23]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))

('RMSLE Score:', 0.30244135602997674)


In [24]:
configLag0Target1DeleteColumnsFalse = ConfigElements(1,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag1", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [25]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.069195
            681747     3306     2281       7               7.317818

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,True,1.418015,NaN,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,True,1.279644,NaN,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384


Count of missing numbers after SPn0_mean in validation part 1 in column Lag1 is 1215


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,True,1.418015,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,True,1.279644,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,1.897133


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384


In [26]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [27]:
FE.SaveDataFrameToHdf('both')

In [28]:
configLag0Target1DeleteColumnsFalse = ConfigElements(2,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag2", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [29]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.069195
            681747     3306     2281       7               7.317818

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,True,1.418015,1.418015,NaN,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,True,1.279644,1.279644,NaN,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,1.897133,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,SPClRACh0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,NaN,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,SPn0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,2.109973
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,1.035384,1.257738


Count of missing numbers after SPn0_mean in validation part 1 in column Lag2 is 1215
Count of missing numbers after SPn0_mean in validation part 2 in column Lag2 is 17824


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,True,1.418015,1.418015,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,True,1.279644,1.279644,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,1.897133,1.897133


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,1.035384


In [30]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [31]:
FE.SaveDataFrameToHdf('both')

In [32]:
configLag0Target1DeleteColumnsFalse = ConfigElements(3,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag3", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [33]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

/home/cankoklu/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.029042
            681747     3306     2281       7               7.332602

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,120,2,709,4,True,1.418015,1.418015,1.418015,NaN,1.599232
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,135,2,712,4,True,1.279644,1.279644,1.279644,NaN,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,1.897133,1.897133,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3,SPn0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441,2.109331
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,1.035384,1.037599,1.261321


Count of missing numbers after SPn0_mean in validation part 1 in column Lag3 is 47405
Count of missing numbers after SPn0_mean in validation part 2 in column Lag3 is 69090


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,True,1.418015,1.418015,1.418015,1.599232
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,True,1.279644,1.279644,1.279644,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,Lag3
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.897133,1.897133,1.897133,1.917298


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.035384,1.035384,1.037599


In [34]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [35]:
FE.SaveDataFrameToHdf('both')

In [36]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag1))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag1))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag1))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag2))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag2))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag2))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag3))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag3))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag3))

('RMSLE Score:', 0.30244135602997674)
('RMSLE Score:', 0.30244135602997674)
('RMSLE Score:', 0.30244135602997674)
('RMSLE Score:', 0.32334618842645518)
